# Make a catalog of SEDs

In [9]:
# Imports
from SEDkit import sed, spectrum, synphot
import astropy.units as q
import numpy as np
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

First we initialize an empty catalog.

In [10]:
# Make an SED catalog
cat = sed.SEDCatalog()

Now let's make a few SEDs

In [11]:
# Make an SED
s1 = sed.SED(parallax=(1.7474*q.mas, 0.0501*q.mas), spectral_type='M0V', name='2MASS 261832886')
s1.add_photometry_file(resource_filename('SEDkit', 'data/M0V_photometry.txt'))
s1.make_sed()
cat.add_SED(s1)

Make this handle asymmetric uncertainties!
Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)
Setting radius to (<Quantity 0.8786209573091851 solRad>, <Quantity 0.06782337214316517 solRad>)


In [12]:
# Make 30 other SEDs (I'll just change the parallax)
for i in range(10):
    s2 = copy.copy(s1)
    s2.parallax = abs(np.random.normal(loc=1.7474, scale=0.1))*q.mas, 0.1*q.mas
    s2.name += str(i)
    s2.make_sed()
    cat.add_SED(s2)

Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!


Then the aggregated results look like this:

In [5]:
cat.results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,float64,float64,object,object,float64,float64,object,object,object,object,object,object,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2MASS 261832886,6000.0,4000.0,572.28,16.41,1.7474,0.0501,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.66179055973561e+32,2.1287295997847215e+31,-1.019,0.025,7.304,0.033,4.5,0.07,0.8896913720017506,0.0,3424.0,141.0,<SEDkit.sed.SED object at 0x11256bd68>,20.486,0.073,67092.551,3847.73,17.779,0.006,58227.007,3339.293,16.431,0.004,53812.248,3086.108,15.843,0.004,51886.522,2975.669,15.514,0.005,50809.032,2913.875,14.391,0.029,47131.158,2702.951,13.677,0.032,44792.777,2568.846,13.535,0.034,44327.72,2542.176,13.55,0.027,44376.846,2544.993,13.581,0.035,44478.372,2550.815,12.884,0.0,42195.667,2419.902
2MASS 2618328860,6000.0,4000.0,541.21,29.29,1.8477279880254265,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.274974997556934e+32,3.5584695449337295e+31,-1.068,0.047,7.425,0.06,4.5,0.07,0.8896913720017506,0.0,3330.0,157.0,<SEDkit.sed.SED object at 0x113096828>,20.486,0.073,60005.187,6494.899,17.779,0.006,52076.16,5636.669,16.431,0.004,48127.757,5209.298,15.843,0.004,46405.456,5022.878,15.514,0.005,45441.788,4918.571,14.391,0.029,42152.428,4562.535,13.677,0.032,40061.063,4336.168,13.535,0.034,39645.134,4291.148,13.55,0.027,39689.07,4295.903,13.581,0.035,39779.871,4305.732,12.884,0.0,37738.301,4084.754
2MASS 2618328861,6000.0,4000.0,509.53,25.96,1.9625789482121032,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,2.902791766238549e+32,2.9707501192020326e+31,-1.12,0.044,7.556,0.056,4.5,0.07,0.8896913720017506,0.0,3231.0,149.0,<SEDkit.sed.SED object at 0x1130f5048>,20.486,0.073,53185.921,5419.531,17.779,0.006,46157.986,4703.398,16.431,0.004,42658.297,4346.788,15.843,0.004,41131.727,4191.234,15.514,0.005,40277.574,4104.197,14.391,0.029,37362.032,3807.11,13.677,0.032,35508.34,3618.223,13.535,0.034,35139.678,3580.657,13.55,0.027,35178.621,3584.625,13.581,0.035,35259.104,3592.827,12.884,0.0,33449.547,3408.436
2MASS 2618328862,6000.0,4000.0,614.84,37.8,1.6264309638591052,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.2266918996459e+32,5.212628905992513e+31,-0.957,0.054,7.148,0.067,4.5,0.07,0.8896913720017506,0.0,3549.0,175.0,<SEDkit.sed.SED object at 0x1130d2be0>,20.486,0.073,77442.862,9522.284,17.779,0.006,67209.638,8264.018,16.431,0.004,62113.818,7637.442,15.843,0.004,59891.012,7364.128,15.514,0.005,58647.299,7211.203,14.391,0.029,54402.042,6689.211,13.677,0.032,51702.92,63

We can also plot the results by passing the `plot` method two column names.

In [6]:
show(cat.plot('distance', 'Teff'))

Or plot a CMD! You can also pass the difference of two magnitudes as a string like so.

In [7]:
show(cat.plot('2MASS.J-2MASS.Ks', 'M_2MASS.J'))

We can then retrieve the SED for a particular source like so:

In [8]:
my_sed = cat.get_SED('2MASS 261832886')
fig = my_sed.plot()

No SEDs named 2MASS 261832886


AttributeError: 'NoneType' object has no attribute 'plot'

And save and load from file.

In [ ]:
cat.save('/Users/jfilippazzo/Desktop/test.p')

In [ ]:
new = sed.SEDCatalog().load('/Users/jfilippazzo/Desktop/test.p')

The catalog can also be filtered by providing a column name and value...

In [39]:
cat.filter('name', "'2MASS 2618328861'").results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,float64,float64,object,object,float64,float64,object,object,object,object,object,object,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2MASS 2618328861,6000.0,4000.0,598.98,35.88,1.6695005640755598,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.01144652103885e+32,4.820994360627056e+31,-0.98,0.052,7.205,0.066,4.5,0.07,0.8896913720017506,0.0,3503.0,171.0,<SEDkit.sed.SED object at 0x11af7de48>,20.486,0.073,73499.064,8805.458,17.779,0.006,63786.97,7641.913,16.431,0.004,58950.656,7062.505,15.843,0.004,56841.047,6809.766,15.514,0.005,55660.67,6668.352,14.391,0.029,51631.604,6185.656,13.677,0.032,49069.936,5878.759,13.535,0.034,48560.472,5817.723,13.55,0.027,48614.289,5824.17,13.581,0.035,48725.51,5837.495,12.884,0.0,46224.834,5537.905


...or a wildcard search...

In [43]:
cat.filter('name', "*328861").results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,float64,float64,object,object,float64,float64,object,object,object,object,object,object,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2MASS 2618328861,6000.0,4000.0,598.98,35.88,1.6695005640755598,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.01144652103885e+32,4.820994360627056e+31,-0.98,0.052,7.205,0.066,4.5,0.07,0.8896913720017506,0.0,3503.0,171.0,<SEDkit.sed.SED object at 0x11af7de48>,20.486,0.073,73499.064,8805.458,17.779,0.006,63786.97,7641.913,16.431,0.004,58950.656,7062.505,15.843,0.004,56841.047,6809.766,15.514,0.005,55660.67,6668.352,14.391,0.029,51631.604,6185.656,13.677,0.032,49069.936,5878.759,13.535,0.034,48560.472,5817.723,13.55,0.027,48614.289,5824.17,13.581,0.035,48725.51,5837.495,12.884,0.0,46224.834,5537.905


... or an argument with an operator...

In [41]:
cat.filter('logg', '>=4.5').results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,float64,float64,object,object,float64,float64,object,object,object,object,object,object,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2MASS 261832886,6000.0,4000.0,572.28,16.41,1.7474,0.0501,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.66179055973561e+32,2.1287295997847215e+31,-1.019,0.025,7.304,0.033,4.5,0.07,0.8896913720017506,0.0,3424.0,141.0,<SEDkit.sed.SED object at 0x11a1bdac8>,20.486,0.073,67092.551,3847.73,17.779,0.006,58227.007,3339.293,16.431,0.004,53812.248,3086.108,15.843,0.004,51886.522,2975.669,15.514,0.005,50809.032,2913.875,14.391,0.029,47131.158,2702.951,13.677,0.032,44792.777,2568.846,13.535,0.034,44327.72,2542.176,13.55,0.027,44376.846,2544.993,13.581,0.035,44478.372,2550.815,12.884,0.0,42195.667,2419.902
2MASS 2618328860,6000.0,4000.0,572.64,32.79,1.7463009673899073,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.6663990004665696e+32,4.2133096135621395e+31,-1.019,0.05,7.303,0.063,4.5,0.07,0.8896913720017506,0.0,3425.0,164.0,<SEDkit.sed.SED object at 0x11afe1ef0>,20.486,0.073,67176.988,7693.259,17.779,0.006,58300.287,6676.678,16.431,0.004,53879.972,6170.454,15.843,0.004,51951.822,5949.638,15.514,0.005,50872.977,5826.086,14.391,0.029,47190.474,5404.358,13.677,0.032,44849.149,5136.224,13.535,0.034,44383.508,5082.898,13.55,0.027,44432.695,5088.531,13.581,0.035,44534.349,5100.173,12.884,0.0,42248.771,4838.423
2MASS 2618328861,6000.0,4000.0,598.98,35.88,1.6695005640755598,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.01144652103885e+32,4.820994360627056e+31,-0.98,0.052,7.205,0.066,4.5,0.07,0.8896913720017506,0.0,3503.0,171.0,<SEDkit.sed.SED object at 0x11af7de48>,20.486,0.073,73499.064,8805.458,17.779,0.006,63786.97,7641.913,16.431,0.004,58950.656,7062.505,15.843,0.004,56841.047,6809.766,15.514,0.005,55660.67,6668.352,14.391,0.029,51631.604,6185.656,13.677,0.032,49069.936,5878.759,13.535,0.034,48560.472,5817.723,13.55,0.027,48614.289,5824.17,13.581,0.035,48725.51,5837.495,12.884,0.0,46224.834,5537.905
2MASS 2618328862,6000.0,4000.0,569.45,32.43,1.7560932438236598,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.62566403281865e+32,4.143994293920703e+31,-1.024,0.05,7.315,0.063,4.5,0.07,0.8896913720017506,0.0,3416.0,164.0,<SEDkit.sed.SED object at 0x11afbc358>,20.486,0.073,66430.629,7566.408,17.779,0.006,57652.55,6566.59,16.431,0.004,53281.346,6068.712,15.843,0.004,51374.619,5851.537,15.514,0.005,50307.76,5730.023,14.391,0.029,46666.171,5315.248,13.677,0.032,44350.86,5051.

... or even a tuple of arguments...

In [44]:
cat.filter('M_SDSS.r', ('>50000','<60000')).results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,float64,float64,object,object,float64,float64,object,object,object,object,object,object,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2MASS 261832886,6000.0,4000.0,572.28,16.41,1.7474,0.0501,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.66179055973561e+32,2.1287295997847215e+31,-1.019,0.025,7.304,0.033,4.5,0.07,0.8896913720017506,0.0,3424.0,141.0,<SEDkit.sed.SED object at 0x11a1bdac8>,20.486,0.073,67092.551,3847.73,17.779,0.006,58227.007,3339.293,16.431,0.004,53812.248,3086.108,15.843,0.004,51886.522,2975.669,15.514,0.005,50809.032,2913.875,14.391,0.029,47131.158,2702.951,13.677,0.032,44792.777,2568.846,13.535,0.034,44327.72,2542.176,13.55,0.027,44376.846,2544.993,13.581,0.035,44478.372,2550.815,12.884,0.0,42195.667,2419.902
2MASS 2618328860,6000.0,4000.0,572.64,32.79,1.7463009673899073,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.6663990004665696e+32,4.2133096135621395e+31,-1.019,0.05,7.303,0.063,4.5,0.07,0.8896913720017506,0.0,3425.0,164.0,<SEDkit.sed.SED object at 0x11afe1ef0>,20.486,0.073,67176.988,7693.259,17.779,0.006,58300.287,6676.678,16.431,0.004,53879.972,6170.454,15.843,0.004,51951.822,5949.638,15.514,0.005,50872.977,5826.086,14.391,0.029,47190.474,5404.358,13.677,0.032,44849.149,5136.224,13.535,0.034,44383.508,5082.898,13.55,0.027,44432.695,5088.531,13.581,0.035,44534.349,5100.173,12.884,0.0,42248.771,4838.423
2MASS 2618328861,6000.0,4000.0,598.98,35.88,1.6695005640755598,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.01144652103885e+32,4.820994360627056e+31,-0.98,0.052,7.205,0.066,4.5,0.07,0.8896913720017506,0.0,3503.0,171.0,<SEDkit.sed.SED object at 0x11af7de48>,20.486,0.073,73499.064,8805.458,17.779,0.006,63786.97,7641.913,16.431,0.004,58950.656,7062.505,15.843,0.004,56841.047,6809.766,15.514,0.005,55660.67,6668.352,14.391,0.029,51631.604,6185.656,13.677,0.032,49069.936,5878.759,13.535,0.034,48560.472,5817.723,13.55,0.027,48614.289,5824.17,13.581,0.035,48725.51,5837.495,12.884,0.0,46224.834,5537.905
2MASS 2618328862,6000.0,4000.0,569.45,32.43,1.7560932438236598,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.62566403281865e+32,4.143994293920703e+31,-1.024,0.05,7.315,0.063,4.5,0.07,0.8896913720017506,0.0,3416.0,164.0,<SEDkit.sed.SED object at 0x11afbc358>,20.486,0.073,66430.629,7566.408,17.779,0.006,57652.55,6566.59,16.431,0.004,53281.346,6068.712,15.843,0.004,51374.619,5851.537,15.514,0.005,50307.76,5730.023,14.391,0.029,46666.171,5315.248,13.677,0.032,44350.86,5051.